# MCS 275 Spring 2022 Homework 12 Solutions

* Course Instructor: David Dumas
* Solutions prepared by: Johnny Joyce

## Instructions:

* Complete the problems below, which ask you to write Python scripts.
* Upload your python code directly to gradescope, i.e. upload the `.py` files containing your work.  (If you upload a screenshot or other file format, you won't get credit.)

### Deadline

This homework assignment must be submitted in Gradescope by **Noon central time on Tuesday 12 April 2022**.

### Collaboration

**Collaboration is prohibited**, and you may only access resources (books, online, etc.) listed below.

### Resources you may consult

These things might be helpful while working on the problems.  Remember that for worksheets,  we don't strictly limit what resources you can consult, so these are only suggestions.

* [Worksheet 12 Solutions](https://www.dumas.io/teaching/2022/spring/mcs275/nbview/worksheets/worksheet12soln.html)
* [Lecture 30](https://www.dumas.io/teaching/2022/spring/mcs275/slides/lecture30.html)
* [Lecture 31](https://www.dumas.io/teaching/2022/spring/mcs275/slides/lecture31.html)
* [Lecture 32](https://www.dumas.io/teaching/2022/spring/mcs275/slides/lecture32.html)
* [SQLite Tutorial (sqlitetutorial.net)](https://www.sqlitetutorial.net/)
* [SQLite language documentation](https://www.sqlite.org/lang.html)

### Point distribution

This homework assignment has two problems.  The grading breakdown is:

| Points | Item |
| --- | --- |
| 2 | Autograder |
| 4 | Problem 2 |
| 4 | Problem 3 |
| **10** | Total |

The part marked "autograder" reflects points assigned to your submission based on some simple automated checks for Python syntax, etc..  The result of these checks is shown immediately after you submit.

### What to do if you're stuck

Ask your instructor or TA a question by email, in office hours, or on discord.

## ( No problem 1 as usual )

## Problem 2

### Tutorial: How to list files and get modification time

For use in the rest of this problem, here's a demonstration of how to list all the files in the current directory, and to get the time (in seconds in Jan 1 1970) when the file was most recently changed.  You can find more about `os.listdir` and `os.getmtime` in the [documentation of the `os` module](https://docs.python.org/3/library/os.html).

In [10]:
import os

for fn in os.listdir():
    print("Filename={}\nLastModified={}\n".format(fn,int(os.path.getmtime(fn))))

Filename=homework11soln.ipynb
LastModified=1649258369

Filename=homework12.ipynb
LastModified=1649357087

Filename=homework10.ipynb
LastModified=1647552323

Filename=homework11.ipynb
LastModified=1647381255

Filename=homework10soln.ipynb
LastModified=1648583672



### The actual task

Now, using the code above as a starting point, write a new program called `hwk12prob2.py` that examines files in the current directory and stores info about their last modified times and how often they've changed in a database.

Specifically, the program must do the following:
* Open a SQLite database file called `lastmod.db`
* Create a table called `files` as follows **if it doesn't already exist**:
    * Column `name` of type TEXT (i.e. a Python string)
    * Column `lastmod` of type INTEGER
    * Column `revcount` of type INTEGER
* Look at every file in the current directory, and do the following for each one:
    * If the filename is `lastmod.db`, just continue to the next iteration of the loop (skipping everything below for this file)
    * If there is **already a row** in the table `files` that has the same `name` as this file, then:
        * Check whether the `lastmod` value for that row is equal to the last modified time of this file.
            * If they are the same: Do nothing
            * If they are different: Update the `lastmod` value for that row to the last modified time of this file, and also increase the value of `revcount` in that row by 1
    * Otherwise (i.e. if there is **no row** in the table `files` that has the same `name` as this file):
        * Create a new row in table `files` using this file's name as `name`, using its last modified time as `lastmod`, and setting `revcount` to 1

This means that if you run `hwk12prob2.py` in a given directory on a regular basis (say, every hour or every few minutes), it will notice every time a file has changed and keep a count of how many different versions it has seen thus far.  The database will also track the most recent modification time of every file, including any that have since been deleted.

Test your program by running it in the directory where you're working on your assignment.  Make a modification to a file in that directory, and run `hwk12prob2.py` again.  Now if you open `lastmod.db` in the SQLite command line shell and run `SELECT * FROM files;` it should show all the files, most of which will have `revcount` equal to 1, but the file you modified should have `revcount` equal to 2.

# Solution

In [18]:
import sqlite3
import os

con = sqlite3.connect("lastmod.db")

con.execute("CREATE TABLE IF NOT EXISTS files (name TEXT, lastmod INT, revcount INT);")

for filename in os.listdir(): # Iterate over every file
    
    if filename == "lastmod.db": # Skip database file itself
        continue

    c = con.execute("SELECT * FROM files WHERE name = ?;", (filename,))
    row = c.fetchone()
    
    if row is not None: # If row is already in table
        
        lastmod_os = os.path.getmtime(filename) # Last modification according to OS
        lastmod_db = row[1] # Last modification according to database
        revcount = row[2]
        
        if int(lastmod_os) != int(lastmod_db):
            con.execute("UPDATE files SET lastmod = ?, revcount = ? WHERE name = ?;", (int(lastmod_os), revcount+1, filename))
            # Print statement not required by question, but is helpful debugging or user experience.
            print("Updated row for file {}. Previous lastmod: {}. New lastmod: {}".format(filename, int(lastmod_db), int(lastmod_os)))
    
    else: # If row is not in table
        
        lastmod_os = os.path.getmtime(filename) # last modification according to OS
        con.execute("INSERT INTO files VALUES (?, ?, ?);", (filename, int(lastmod_os), 1))

con.commit()
con.close()

Updated row for file homework12soln.ipynb. Previous lastmod: 1649364942. New lastmod: 1649364943


### Code snippet to check contents of database:

***Note from Johnny:*** This was run in a folder where I put a bunch of my MCS 275 stuff 🙂 Notice that `homework12soln.ipynb` (i.e. this file you're currently reading) has `revcount` higher than 1 because I intentionally saved and re-ran it many times.

In [19]:
import sqlite3

con = sqlite3.connect("lastmod.db")

c = con.execute("SELECT * FROM files;")

for row in c:
    print(row)
    
con.close()

('.ipynb_checkpoints', 1649362433, 1)
('h.json', 1647452077, 1)
('homework10soln.ipynb', 1648415738, 1)
('homework11soln.ipynb', 1648791499, 1)
('homework11work.ipynb', 1648790666, 1)
('homework12.ipynb', 1649362286, 1)
('homework12soln.ipynb', 1649364943, 13)
('homework4soln.ipynb', 1643942709, 1)
('homework5soln.ipynb', 1644975097, 1)
('homework6soln.ipynb', 1645587381, 1)
('homework7soln.ipynb', 1646156755, 1)
('homework7work.ipynb', 1645816077, 1)
('homework8soln.ipynb', 1646708033, 1)
('homework9soln.ipynb', 1647360818, 1)
('images', 1648416695, 1)
('intsplit.py', 1645136763, 1)
('mcs 572.txt', 1647289717, 1)
('mcs275revealjs.zip', 1646625164, 1)
('mcs275_ml_lecture.txt', 1647224242, 1)
('neural-network.ipynb', 1648665662, 1)
('neural-network2.ipynb', 1648659861, 1)
('project2ans.py', 1644987505, 1)
('project2ans1.py', 1645467873, 1)
('project3.ipynb', 1646343786, 1)
('roomtemp.csv', 1648789159, 1)
('sample.txt', 1644202965, 1)
('slides', 1648097817, 1)
('slides-lec31', 1648671057

## Problem 3

### Show that you've read the feedback on Project 3

By now you've received and read your graded Project 3 submission (the PDF shared with you on google drive, for which I also emailed you a link on 3 April). Hopefully, you have also read through the solution I posted.

As a way to give you some credit for reading the feedback on Project 3, please answer this question:

*At the bottom of the last or second to last page of the PDF of your Project 3 feedback, I included a small drawing labeled "This is just a drawing, not a comment on your work".  Please describe that drawing briefly.*

*(You don't need to be very detailed, e.g. "three green triangles" or "an orange cube" would be enough detail.)*

Put your answer in a text file called `hwk12prob3.txt` and upload it to Gradescope.

### Optional: Ask questions

If you have questions about your project 3 feedback, the solution, or anything else, feel free to include them in `hwk12prob3.txt`.